# Count-3: if patient count = 0, all numerics = 0

Description: check if all the rows will have all other numeric values should be zero when bene count is zero. 

Starting Author: Amy Jin (amy@careset.com)

Date: April 16th, 2018

https://docs.google.com/spreadsheets/d/1IYg01IpssJaWHo6KxO4_dSDgXtYNFy41S5cIHFLvlGQ/edit#gid=604789549

## Connection to Parenthood Server

In [1]:
# Packages import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import mysql.connector
import sshtunnel
import pureyaml

# Handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml");
config = pureyaml.load(config_file.read());

# Argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# Argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

# Connect to Parenthood server
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!


## Printing out Columns Name

In [2]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) db_name:                database name in server
# 2）table_name:             table name
# --------------------------------------- Outputs: --------------------------------------
# 1) Fuction outputs:        print out all the column names in the table


def print_col_names(db_name, table_name):
    #table1 = str(db_name) + '.' + str(table_name)
    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        
        # MySQL query all column names
        query = ('''
                SHOW COLUMNS FROM {db}.{table};
        '''.format(db = db_name, table = table_name))

        cur.execute(query)
        print ('Column names of {}.{} are:'.format(db_name, table_name), '\n')
        for row in cur.fetchall():
            for i in range(0,len(row)):
                print (str(row[i]), end=", ")
            print ('\n')
            
        cur.close()
        connection.close()      

In [4]:
print_col_names('_amy', 'test_data_good')

Column names of _amy.test_data_good are: 

setting, text, YES, , None, , 

hcpcs_cd, text, YES, , None, , 

cnt_rfr_physn_npi, int(11), YES, , None, , 

cnt_pnpi, int(11), YES, , None, , 

cnt_onpi, int(11), YES, , None, , 

cnt_tax_num, int(11), YES, , None, , 

cnt_prvdr_spclty, int(11), YES, , None, , 

cnt_prvdr_state_cd, int(11), YES, , None, , 

cnt_prvdr_zip, int(11), YES, , None, , 

cnt_fi_num, int(11), YES, , None, , 

npi_class, int(11), YES, , None, , 

hcpcs_cd.1, text, YES, , None, , 

cnt_bene_id, int(11), YES, , None, , 

cnt_clm_id, int(11), YES, , None, , 

cnt_star, int(11), YES, , None, , 

icd_dgns_cd, text, YES, , None, , 

npi_dest, bigint(20), YES, , None, , 

patient_count, int(11), YES, , None, , 

transaction_count, int(11), YES, , None, , 

same_day_patient_count, int(11), YES, , None, , 

same_day_transaction_count, int(11), YES, , None, , 

average_day_wait, double, YES, , None, , 

std_day_wait, double, YES, , None, , 

npi, bigint(20), YES, , None, , 

n

## Test Function

In [25]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) db_name:                database name in server
# 2）table_name:             table name
# 3) replace row 19 - 22 with all numeric columns in the above function expect for patient count
# --------------------------------------- Outputs: --------------------------------------
# 1) Test result:            PASS/FAIL
# 2) If FAIL, the test will print out all the rows that have none-zero volume when patient
# count is zero.


def count_3(db_name, table_name):
    
    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        
        query = ('''
                SELECT *
                FROM {db}.{table}
                WHERE (cnt_bene_id = 0 AND cnt_clm_id <> 0)
                OR (cnt_bene_id = 0 AND cnt_star <>0)
                  LIMIT 1;
        '''.format(db = db_name, table = table_name))

        cur.execute(query)
        query = list(sum(cur.fetchall(), ()))

        
        if not len(query):
            print ("Count-1 test result for {}.{} : PASS".format(db_name, table_name))
        else:
            print ("Count-1 test result for {}.{} : FAIL".format(db_name, table_name) + '\n')
            print ("The following rows fail the test:" + '\n')
            
            # MySQL to query failed rows
            query = ('''
                    SELECT *
                    FROM {db}.{table}
                    WHERE (cnt_bene_id = 0 AND cnt_clm_id <> 0)
                        OR (cnt_bene_id = 0 AND cnt_star <>0);
            '''.format(db = db_name, table = table_name))

            cur.execute(query)
            
            for row in cur.fetchall():
                for i in range(0,len(row)):
                    print (str(row[i]), end=", ")
                print ('\n')
            
        cur.close()
        connection.close()      

## Test Example

inst_ndc.inst_ndc_monthly_RIFQ2014

In [26]:
count_3('_amy', 'test_data_good')

Count-1 test result for _amy.test_data_good : PASS


In [27]:
count_3('_amy', 'test_data_bad1')

Count-1 test result for _amy.test_data_bad1 : FAIL

The following rows fail the test:

CAR, F0, 10, 1, 536, 3167, 1, 53, 182, 0, 1, 0, 13.0, 0, U, 1467401240, 8, 14, 1, 0, 1.0, 52.072, 2062779183, 2147483648, 

CAR, 11421, 3, 1, 1, 95, 17, 4, 10, 0, 1, 0, 6.0, 0, 11111111, 3585565578, 41, 25, 0, 0, 20.0, 81.756, hello, , 

CAR, 15574, 63927, 17868, 717, 2603, 9, 54, 878, 0, 1, 0, 7.0, 0, C459, 1828263176, 6, 14, 0, 0, 39.083, 44.7265, 1485061657, , 

CAR, 21936, 2, 676, 25, 7260, 24, 52, 4604, 0, 1, 0, 10.0, 0, C49A1, 2147483648, 27, 110, 0, 0, 93.5985, 47.8715, hello, , 

CAR, 25446, 1302, 219, 4800, 9, 1, 40, 648, 0, 1, 0, 10.0, 0, U, 688119339, 37, 2, 1, 0, 1.0, 57.83, 246798161, 2147483648, 

CAR, 24340, 494, 136, 275, 5804, 14, 55, 1767, 0, 1, 0, 23.0, 0, , -3616650945, 15, 8, 1, 0, 19.188, 54.9385, hello, 2147483648, 

CAR, 1100P, 91, 9417, 202, 725, 1, 35, 719, 0, 1, 0, 7.0, 0, U, hello, 15, 15, 1, 0, 108.842, 83.0625, -6740599147, 2446230, 

CAR, 21627, 1051, 178, 1607, 101, 44